## **Αναγκαία πακέτα**

In [1]:
!pip install cdlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.6 MB/s eta 0:00:00


In [2]:
!pip install wurlitzer

In [3]:
!pip install leidenalg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00


In [4]:
#imports

import networkx as nx
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from networkx.algorithms.centrality import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
import csv
from cdlib import algorithms
from cdlib import readwrite
from sklearn.cluster import SpectralClustering
import time

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'infomap', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap'}


## **Προετοιμασία μοντέλου απόφασης**

In [5]:
data = pd.read_csv('network_data.csv', index_col= 0)

In [6]:
#model training and evaluation

# Prepare data for training
X = data.drop("Network Type", axis=1)
y = data["Network Type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train.values, y_train)

# Make predictions
y_pred = model.predict(X_test.values)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.975
Precision: 0.975925925925926
Recall: 0.975
F1-score: 0.975038665038665


In [7]:
def extract_features(graph):

    degree_distribution = np.array(list(dict(nx.degree(graph)).values()))
    avg_path_length = nx.average_shortest_path_length(graph)
    diameter = nx.diameter(graph)
    information = list(nx.information_centrality(graph).values())
    closeness = list(nx.closeness_centrality(graph).values())
    harmonic = list(nx.harmonic_centrality(graph).values())
    assortativity = nx.degree_assortativity_coefficient(graph)

    features = [
        np.mean(degree_distribution),
        avg_path_length,
        diameter,
        np.mean(information),
        np.mean(closeness),
        np.mean(harmonic),
        assortativity,
    ]
    return features

## **Αλγόριθμοι community detection**

In [8]:
def girvan_newman_communities(G):
  start = time.time()
  gn_com = nx.algorithms.community.girvan_newman(G)
  com_number = 0
  correct_nodes = 0
  correct_communities = 0
  discarded_nodes = 0
  com_len = []
  com = tuple(sorted(c) for c in next(gn_com))
  for c in com:
    if len(c) <= 20 :
      discarded_nodes = discarded_nodes + len(c)
      continue
    com_len.append(len(c))
    com_number = com_number + 1
    c_features = extract_features(G.subgraph(c))
    pred = model.predict(np.array(c_features).reshape(1, -1))
    if com_number == 1:
     com_types = [pred[0]]
    if pred[0] not in com_types:
      com_types.append(pred[0])
    correct_nodes_local = 0
    for i in c:
      if str(i)[:3] == pred[0][:3]:
        correct_nodes_local = correct_nodes_local + 1
    correct_nodes = correct_nodes + correct_nodes_local
    if correct_nodes_local/len(c) > 0.7:
      correct_communities = correct_communities + 1
  print("Number of communities:", com_number)
  print("Number of different community types:", len(com_types))
  print(com_types)
  print("Size of communities:", sorted(com_len, reverse = True))
  print("Number of discarded nodes:",  discarded_nodes)
  print("Correctly classified nodes:", correct_nodes, "(out of", G.number_of_nodes() - discarded_nodes, "nodes)")
  print("Correctly classified communities:", correct_communities)
  print("Modularity:", nx.algorithms.community.modularity(G, com))
  end = time.time()
  print("Runtime: ", end - start)

In [21]:
def walktrap_communities(G):
  start = time.time()
  walk_com = algorithms.walktrap(G)
  com = walk_com.communities
  com_number = 0
  correct_nodes = 0
  correct_communities = 0
  discarded_nodes = 0
  com_len = []
  for c in com:
    if len(c) <= 20:
      discarded_nodes = discarded_nodes + len(c)
      continue
    com_len.append(len(c))
    com_number = com_number + 1
    c_features = extract_features(G.subgraph(c))
    pred = model.predict(np.array(c_features).reshape(1, -1))
    if com_number == 1:
      com_types = [pred[0]]
    if pred[0] not in com_types:
      com_types.append(pred[0])
    correct_nodes_local = 0
    for i in c:
      if str(i)[:3] == pred[0][:3]:
        correct_nodes_local = correct_nodes_local + 1
    correct_nodes = correct_nodes + correct_nodes_local
    if correct_nodes_local / len(c) > 0.7:
      correct_communities = correct_communities + 1
  print("Number of communities:", com_number)
  print("Number of different community types:", len(com_types))
  print(com_types)
  print("Size of communities:", sorted(com_len, reverse=True))
  print("Number of discarded nodes:", discarded_nodes)
  print("Correctly classified nodes:", correct_nodes, "(out of", G.number_of_nodes() - discarded_nodes, "nodes)")
  print("Correctly classified communities:", correct_communities)
  print("Modularity:", nx.algorithms.community.modularity(G, com))
  end = time.time()
  print("Runtime: ", end - start)

In [10]:
def infomap_communities(G):
  start = time.time()
  info_com = algorithms.infomap(G)
  com = info_com.communities
  com_number = 0
  correct_nodes = 0
  correct_communities = 0
  discarded_nodes = 0
  com_len = []
  for c in com:
    if len(c) <= 20:
      discarded_nodes = discarded_nodes + len(c)
      continue
    com_len.append(len(c))
    com_number = com_number + 1
    c_features = extract_features(G.subgraph(c))
    pred = model.predict(np.array(c_features).reshape(1, -1))
    if com_number == 1:
      com_types = [pred[0]]
    if pred[0] not in com_types:
      com_types.append(pred[0])
    correct_nodes_local = 0
    for i in c:
      if str(i)[:3] == pred[0][:3]:
        correct_nodes_local = correct_nodes_local + 1
    correct_nodes = correct_nodes + correct_nodes_local
    if correct_nodes_local / len(c) > 0.7:
      correct_communities = correct_communities + 1
  print("Number of communities:", com_number)
  print("Number of different community types:", len(com_types))
  print(com_types)
  print("Size of communities:", sorted(com_len, reverse=True))
  print("Number of discarded nodes:", discarded_nodes)
  print("Correctly classified nodes:", correct_nodes, "(out of", G.number_of_nodes() - discarded_nodes, "nodes)")
  print("Correctly classified communities:", correct_communities)
  print("Modularity:", nx.algorithms.community.modularity(G, com))
  end = time.time()
  print("Runtime: ", end - start)

In [11]:
def leiden_communities(G):
  start = time.time()
  leiden_com = algorithms.leiden(G)
  com = leiden_com.communities
  com_number = 0
  correct_nodes = 0
  correct_communities = 0
  discarded_nodes = 0
  com_len = []
  com_types = []
  for c in com:
    if len(c) <= 20 :
      discarded_nodes = discarded_nodes + len(c)
      continue
    com_len.append(len(c))
    com_number = com_number + 1
    c_features = extract_features(G.subgraph(c))
    pred = model.predict(np.array(c_features).reshape(1, -1))
    if com_number == 1:
      com_types = [pred[0]]
    if pred[0] not in com_types:
      com_types.append(pred[0])
    correct_nodes_local = 0
    for i in c:
      if str(i)[:3] == pred[0][:3]:
        correct_nodes_local = correct_nodes_local + 1
    correct_nodes = correct_nodes + correct_nodes_local
    if correct_nodes_local/len(c) > 0.7:
      correct_communities = correct_communities + 1

  print("Number of communities:", com_number)
  print("Number of different community types:", len(com_types))
  print(com_types)
  print("Size of communities:", sorted(com_len, reverse = True))
  print("Number of discarded nodes:",  discarded_nodes)
  print("Correctly classified nodes:", correct_nodes, "(out of", G.number_of_nodes() - discarded_nodes, "nodes)")
  print("Correctly classified communities:", correct_communities)
  print("Modularity:", nx.algorithms.community.modularity(G, com))
  end = time.time()
  print("Runtime: ", end - start)

In [12]:
def spectral_communities(G, k):
  start = time.time()
  am = nx.to_numpy_array(G)
  spectral = SpectralClustering(n_clusters=k, random_state=0, affinity='precomputed').fit(am)
  clabels=spectral.labels_
  com = [[] for _ in range(k)]
  node_list = list(G.nodes())
  for node_index, label in enumerate(clabels):
      com[label].append(node_list[node_index])
  com_number = 0
  correct_nodes = 0
  correct_communities = 0
  discarded_nodes = 0
  com_len = []
  for c in com:
    if (len(c) <= 20) or not nx.is_connected(G.subgraph(c)) :
      discarded_nodes = discarded_nodes + len(c)
      continue
    com_len.append(len(c))
    com_number = com_number + 1
    c_features = extract_features(G.subgraph(c))
    pred = model.predict(np.array(c_features).reshape(1, -1))
    if com_number == 1:
     com_types = [pred[0]]
    if pred[0] not in com_types:
      com_types.append(pred[0])
    correct_nodes_local = 0
    for i in c:
      if str(i)[:3] == pred[0][:3]:
        correct_nodes_local = correct_nodes_local + 1
    correct_nodes = correct_nodes + correct_nodes_local
    if correct_nodes_local/len(c) > 0.7:
      correct_communities = correct_communities + 1
  print("Number of communities:", com_number)
  print("Number of different community types:", len(com_types))
  print(com_types)
  print("Size of communities:", sorted(com_len, reverse = True))
  print("Number of discarded nodes:",  discarded_nodes)
  print("Correctly classified nodes:", correct_nodes, "(out of", G.number_of_nodes() - discarded_nodes, "nodes)")
  print("Correctly classified communities:", correct_communities)
  print("Modularity:", nx.algorithms.community.modularity(G, com))
  end = time.time()
  print("Runtime: ", end - start)

## **Τεχνητά δίκτυα για testing**

### **Απλά Δίκτυα**

In [13]:
er = nx.gnm_random_graph(1401, 2399)
er = nx.relabel_nodes(er, {n: f"Erd_{n}" for n in er.nodes()})
er_nodes = list(er.nodes())

gi = nx.gnp_random_graph(1200, 0.5)
gi = nx.relabel_nodes(gi, {n: f"Gil_{n}" for n in gi.nodes()})
gi_nodes = list(gi.nodes())

rg = nx.random_geometric_graph(1392, 0.3)
rg = nx.relabel_nodes(rg, {n: f"Ran_{n}" for n in rg.nodes()})
rg_nodes = list(rg.nodes())

sw = nx.watts_strogatz_graph(1299, 2, 0.4)
sw = nx.relabel_nodes(sw, {n: f"Wat_{n}" for n in sw.nodes()})
sw_nodes = list(sw.nodes())

sf = nx.barabasi_albert_graph(1519, 6)
sf = nx.relabel_nodes(sf, {n: f"Bar_{n}" for n in sf.nodes()})
sf_nodes = list(sf.nodes())

wa = nx.waxman_graph(1333, 0.6)
wa = nx.relabel_nodes(wa, {n: f"Wax_{n}" for n in wa.nodes()})
wa_nodes = list(wa.nodes())

### **Scalefree και Random Geometric**

In [14]:
sf_rg = nx.compose(sf, rg)

for _ in range(221):
    u = random.choice(sf_nodes)
    v = random.choice(rg_nodes)
    sf_rg.add_edge(u, v)

while not nx.is_connected(sf_rg):
    u = random.choice(sf_nodes)
    v = random.choice(rg_nodes)
    sf_rg.add_edge(u, v)

In [15]:
walktrap_communities(sf_rg)

Number of communities: 5
Number of different community types: 2
['Barabasi Albert', 'Random Geometric']
Size of communities: [1519, 467, 321, 307, 297]
Number of discarded nodes: 0
Correctly classified nodes: 2911 (out of 2911 nodes)
Correctly classified communities: 5
Modularity: 0.4949602710034162
Runtime:  184.10029530525208


In [16]:
infomap_communities(sf_rg)

Number of communities: 5
Number of different community types: 3
['Barabasi Albert', 'Gilbert', 'Random Geometric']
Size of communities: [1519, 407, 388, 323, 274]
Number of discarded nodes: 0
Correctly classified nodes: 2504 (out of 2911 nodes)
Correctly classified communities: 4
Modularity: 0.5177840497077912
Runtime:  144.7040412425995


In [17]:
leiden_communities(sf_rg)

Number of communities: 5
Number of different community types: 2
['Barabasi Albert', 'Random Geometric']
Size of communities: [1519, 388, 379, 331, 294]
Number of discarded nodes: 0
Correctly classified nodes: 2911 (out of 2911 nodes)
Correctly classified communities: 5
Modularity: 0.5183267757904987
Runtime:  135.79667711257935


In [18]:
spectral_communities(sf_rg, 2)

Number of communities: 2
Number of different community types: 2
['Barabasi Albert', 'Random Geometric']
Size of communities: [1519, 1392]
Number of discarded nodes: 0
Correctly classified nodes: 2911 (out of 2911 nodes)
Correctly classified communities: 2
Modularity: 0.0812627075181599
Runtime:  1095.2439177036285


### **Scale-free και Small world**

In [19]:
sf_sw = nx.compose(sf, sw)

for _ in range(402):
    u = random.choice(sf_nodes)
    v = random.choice(sw_nodes)
    sf_sw.add_edge(u, v)

while not nx.is_connected(sf_sw):
    u = random.choice(sf_nodes)
    v = random.choice(sw_nodes)
    sf_sw.add_edge(u, v)

In [22]:
walktrap_communities(sf_sw)

Number of communities: 1
Number of different community types: 1
['Barabasi Albert']
Size of communities: [1581]
Number of discarded nodes: 1237
Correctly classified nodes: 1519 (out of 1581 nodes)
Correctly classified communities: 1
Modularity: 0.17902490351725592
Runtime:  95.49317860603333


In [23]:
infomap_communities(sf_sw)

Number of communities: 1
Number of different community types: 1
['Barabasi Albert']
Size of communities: [1494]
Number of discarded nodes: 1324
Correctly classified nodes: 1494 (out of 1494 nodes)
Correctly classified communities: 1
Modularity: 0.20869314710088374
Runtime:  65.63439965248108


In [24]:
leiden_communities(sf_sw)

Number of communities: 23
Number of different community types: 3
['Barabasi Albert', 'Erdos-Renyi', 'Watts Strogatz']
Size of communities: [348, 313, 311, 189, 156, 135, 110, 104, 104, 95, 89, 83, 81, 77, 76, 76, 76, 69, 69, 68, 67, 65, 38]
Number of discarded nodes: 19
Correctly classified nodes: 2321 (out of 2799 nodes)
Correctly classified communities: 19
Modularity: 0.3592963681661677
Runtime:  13.57032322883606


In [25]:
spectral_communities(sf_sw, 2)

Number of communities: 1
Number of different community types: 1
['Watts Strogatz']
Size of communities: [2808]
Number of discarded nodes: 10
Correctly classified nodes: 1289 (out of 2808 nodes)
Correctly classified communities: 0
Modularity: 0.0016683601889133344
Runtime:  237.19362807273865


### **Small Wolrd και Erdos-Renyi**

In [26]:
er_sw = nx.compose(er, sw)

for _ in range(354):
    u = random.choice(er_nodes)
    v = random.choice(sw_nodes)
    er_sw.add_edge(u, v)

while not nx.is_connected(er_sw):
    u = random.choice(er_nodes)
    v = random.choice(sw_nodes)
    er_sw.add_edge(u, v)

In [27]:
walktrap_communities(er_sw)

Number of communities: 17
Number of different community types: 3
['Watts Strogatz', 'Barabasi Albert', 'Erdos-Renyi']
Size of communities: [595, 75, 56, 55, 50, 41, 35, 34, 30, 30, 30, 28, 28, 27, 23, 23, 22]
Number of discarded nodes: 1518
Correctly classified nodes: 428 (out of 1182 nodes)
Correctly classified communities: 0
Modularity: 0.2656585059339352
Runtime:  14.416789531707764


In [28]:
infomap_communities(er_sw)

Number of communities: 4
Number of different community types: 1
['Barabasi Albert']
Size of communities: [24, 22, 21, 21]
Number of discarded nodes: 2612
Correctly classified nodes: 0 (out of 88 nodes)
Correctly classified communities: 0
Modularity: 0.327152187637336
Runtime:  12.746045589447021


In [29]:
leiden_communities(er_sw)

Number of communities: 22
Number of different community types: 1
['Watts Strogatz']
Size of communities: [236, 175, 170, 136, 125, 124, 122, 116, 115, 115, 114, 114, 110, 108, 108, 107, 107, 103, 102, 101, 101, 91]
Number of discarded nodes: 0
Correctly classified nodes: 1299 (out of 2700 nodes)
Correctly classified communities: 0
Modularity: 0.37918432560137244
Runtime:  8.172728061676025


In [30]:
spectral_communities(er_sw, 2)

Number of communities: 2
Number of different community types: 1
['Watts Strogatz']
Size of communities: [1362, 1338]
Number of discarded nodes: 0
Correctly classified nodes: 1299 (out of 2700 nodes)
Correctly classified communities: 0
Modularity: 0.23759657240783372
Runtime:  100.72625064849854


### **Waxman και Random Geometric**

In [31]:
wa_rg = nx.compose(wa, rg)

for _ in range(189):
    u = random.choice(wa_nodes)
    v = random.choice(rg_nodes)
    wa_rg.add_edge(u, v)

while not nx.is_connected(wa_rg):
    u = random.choice(wa_nodes)
    v = random.choice(rg_nodes)
    wa_rg.add_edge(u, v)

In [32]:
walktrap_communities(wa_rg)

Number of communities: 8
Number of different community types: 2
['Waxman', 'Random Geometric']
Size of communities: [411, 395, 384, 364, 349, 295, 291, 236]
Number of discarded nodes: 0
Correctly classified nodes: 2725 (out of 2725 nodes)
Correctly classified communities: 8
Modularity: 0.5411639573350023
Runtime:  121.56376624107361


In [33]:
infomap_communities(wa_rg)

Number of communities: 8
Number of different community types: 2
['Random Geometric', 'Waxman']
Size of communities: [425, 413, 394, 350, 325, 299, 296, 223]
Number of discarded nodes: 0
Correctly classified nodes: 2725 (out of 2725 nodes)
Correctly classified communities: 8
Modularity: 0.5501644435699319
Runtime:  105.26951360702515


In [34]:
leiden_communities(wa_rg)

Number of communities: 5
Number of different community types: 2
['Waxman', 'Random Geometric']
Size of communities: [1333, 393, 369, 349, 281]
Number of discarded nodes: 0
Correctly classified nodes: 2725 (out of 2725 nodes)
Correctly classified communities: 5
Modularity: 0.5816570557970223
Runtime:  198.9633548259735


In [35]:
spectral_communities(wa_rg, 2)

Number of communities: 2
Number of different community types: 2
['Waxman', 'Random Geometric']
Size of communities: [1392, 1333]
Number of discarded nodes: 0
Correctly classified nodes: 2725 (out of 2725 nodes)
Correctly classified communities: 2
Modularity: 0.28090669911754734
Runtime:  1085.8024227619171


### **Glibert και Waxman**

In [36]:
gi_wa = nx.compose(gi, wa)

for _ in range(333):
    u = random.choice(gi_nodes)
    v = random.choice(wa_nodes)
    gi_wa.add_edge(u, v)

while not nx.is_connected(gi_wa):
    u = random.choice(gi_nodes)
    v = random.choice(wa_nodes)
    gi_wa.add_edge(u, v)

In [37]:
walktrap_communities(gi_wa)

Number of communities: 2
Number of different community types: 2
['Waxman', 'Gilbert']
Size of communities: [1333, 1200]
Number of discarded nodes: 0
Correctly classified nodes: 2533 (out of 2533 nodes)
Correctly classified communities: 2
Modularity: 0.18589140099656504
Runtime:  366.8453686237335


In [38]:
infomap_communities(gi_wa)

Number of communities: 5
Number of different community types: 2
['Gilbert', 'Waxman']
Size of communities: [1200, 376, 334, 319, 304]
Number of discarded nodes: 0
Correctly classified nodes: 2533 (out of 2533 nodes)
Correctly classified communities: 5
Modularity: 0.1623918532720286
Runtime:  240.93437552452087


In [39]:
leiden_communities(gi_wa)

Number of communities: 2
Number of different community types: 2
['Waxman', 'Gilbert']
Size of communities: [1333, 1200]
Number of discarded nodes: 0
Correctly classified nodes: 2533 (out of 2533 nodes)
Correctly classified communities: 2
Modularity: 0.18589140099656504
Runtime:  340.5033848285675


In [40]:
spectral_communities(gi_wa, 2)

Number of communities: 2
Number of different community types: 2
['Waxman', 'Gilbert']
Size of communities: [1333, 1200]
Number of discarded nodes: 0
Correctly classified nodes: 2533 (out of 2533 nodes)
Correctly classified communities: 2
Modularity: 0.18589140099656504
Runtime:  338.5971326828003


## **Τεστ για Girvan Newmann**

### **Απλά Δίκτυα** (Μικρότερα)

In [41]:
er_gn = nx.gnm_random_graph(401, 799)
er_gn = nx.relabel_nodes(er_gn, {n: f"Erd_{n}" for n in er_gn.nodes()})
er_gn_nodes = list(er_gn.nodes())

gi_gn = nx.gnp_random_graph(200, 0.5)
gi_gn = nx.relabel_nodes(gi_gn, {n: f"Gil_{n}" for n in gi_gn.nodes()})
gi_gn_nodes = list(gi_gn.nodes())

rg_gn = nx.random_geometric_graph(392, 0.3)
rg_gn = nx.relabel_nodes(rg_gn, {n: f"Ran_{n}" for n in rg_gn.nodes()})
rg_gn_nodes = list(rg_gn.nodes())

sw_gn = nx.watts_strogatz_graph(299, 2, 0.4)
sw_gn = nx.relabel_nodes(sw_gn, {n: f"Wat_{n}" for n in sw_gn.nodes()})
sw_gn_nodes = list(sw_gn.nodes())

sf_gn = nx.barabasi_albert_graph(519, 6)
sf_gn = nx.relabel_nodes(sf_gn, {n: f"Bar_{n}" for n in sf_gn.nodes()})
sf_gn_nodes = list(sf_gn.nodes())

wa_gn = nx.waxman_graph(333, 0.6)
wa_gn = nx.relabel_nodes(wa_gn, {n: f"Wax_{n}" for n in wa_gn.nodes()})
wa_gn_nodes = list(wa_gn.nodes())

### **Scalefree και Random Geometric**

In [42]:
sf_rg_gn = nx.compose(sf_gn, rg_gn)

for _ in range(57):
    u = random.choice(sf_gn_nodes)
    v = random.choice(rg_gn_nodes)
    sf_rg_gn.add_edge(u, v)

while not nx.is_connected(sf_rg_gn):
    u = random.choice(sf_gn_nodes)
    v = random.choice(rg_gn_nodes)
    sf_rg_gn.add_edge(u, v)

In [43]:
girvan_newman_communities(sf_rg_gn)

Number of communities: 2
Number of different community types: 2
['Barabasi Albert', 'Random Geometric']
Size of communities: [519, 392]
Number of discarded nodes: 0
Correctly classified nodes: 911 (out of 911 nodes)
Correctly classified communities: 2
Modularity: 0.25962226253004395
Runtime:  756.6220574378967


### **Scale-free και Small world**

In [44]:
sf_sw_gn = nx.compose(sf_gn, sw_gn)

for _ in range(42):
    u = random.choice(sf_gn_nodes)
    v = random.choice(sw_gn_nodes)
    sf_sw_gn.add_edge(u, v)

while not nx.is_connected(sf_sw_gn):
    u = random.choice(sf_gn_nodes)
    v = random.choice(sw_gn_nodes)
    sf_sw_gn.add_edge(u, v)

In [45]:
girvan_newman_communities(sf_sw_gn)

Number of communities: 2
Number of different community types: 1
['Watts Strogatz']
Size of communities: [791, 27]
Number of discarded nodes: 0
Correctly classified nodes: 299 (out of 818 nodes)
Correctly classified communities: 1
Modularity: 0.01507498870145775
Runtime:  33.77516412734985


### **Small Wolrd και Erdos-Renyi**

In [46]:
sw_er_gn = nx.compose(sw_gn, er_gn)

for _ in range(34):
    u = random.choice(sw_gn_nodes)
    v = random.choice(er_gn_nodes)
    sw_er_gn.add_edge(u, v)

while not nx.is_connected(sw_er_gn):
    u = random.choice(sw_gn_nodes)
    v = random.choice(er_gn_nodes)
    sw_er_gn.add_edge(u, v)

In [47]:
girvan_newman_communities(sw_er_gn)

Number of communities: 1
Number of different community types: 1
['Watts Strogatz']
Size of communities: [698]
Number of discarded nodes: 2
Correctly classified nodes: 297 (out of 698 nodes)
Correctly classified communities: 0
Modularity: 0.0011742236903907683
Runtime:  17.52203869819641


### **Waxman και Random Geometric**

In [48]:
wa_rg_gn = nx.compose(wa_gn, rg_gn)

for _ in range(61):
    u = random.choice(wa_gn_nodes)
    v = random.choice(rg_gn_nodes)
    wa_rg_gn.add_edge(u, v)

while not nx.is_connected(wa_rg_gn):
    u = random.choice(wa_gn_nodes)
    v = random.choice(rg_gn_nodes)
    wa_rg_gn.add_edge(u, v)

In [49]:
girvan_newman_communities(wa_rg_gn)

Number of communities: 2
Number of different community types: 2
['Waxman', 'Random Geometric']
Size of communities: [392, 333]
Number of discarded nodes: 0
Correctly classified nodes: 725 (out of 725 nodes)
Correctly classified communities: 2
Modularity: 0.22799838040009607
Runtime:  590.636376619339


### **Glibert και Waxman**

In [50]:
gi_wa_gn = nx.compose(gi_gn, wa_gn)

for _ in range(53):
    u = random.choice(gi_gn_nodes)
    v = random.choice(wa_gn_nodes)
    gi_wa_gn.add_edge(u, v)

while not nx.is_connected(gi_wa_gn):
    u = random.choice(gi_gn_nodes)
    v = random.choice(wa_gn_nodes)
    gi_wa_gn.add_edge(u, v)

In [51]:
girvan_newman_communities(gi_wa_gn)

Number of communities: 2
Number of different community types: 2
['Gilbert', 'Waxman']
Size of communities: [333, 200]
Number of discarded nodes: 0
Correctly classified nodes: 533 (out of 533 nodes)
Correctly classified communities: 2
Modularity: 0.323879863162281
Runtime:  237.0777268409729


## **Εφαρμογή σε πραγματικά δίκτυα**

### **football**

In [52]:
football=nx.read_gml(r"football.gml")

In [54]:
leiden_communities(football)

Number of communities: 0
Number of different community types: 0
[]
Size of communities: []
Number of discarded nodes: 115
Correctly classified nodes: 0 (out of 0 nodes)
Correctly classified communities: 0
Modularity: 0.6045695626834571
Runtime:  0.02100849151611328


### **GOT**

In [56]:
got=nx.Graph()
with open('got_s5.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    next(csv_reader)
    for row in csv_reader:
        got.add_edge(row[0],row[1])
        line_count += 1

In [57]:
leiden_communities(got)

Number of communities: 2
Number of different community types: 1
['Barabasi Albert']
Size of communities: [35, 23]
Number of discarded nodes: 61
Correctly classified nodes: 0 (out of 58 nodes)
Correctly classified communities: 0
Modularity: 0.6701165697377818
Runtime:  0.08339786529541016


### **email-eu**

In [58]:
#email-Eu-core network
eu=nx.read_edgelist("email-Eu-core.txt",create_using=nx.Graph())
#selecting the largest connected component
bc = max(nx.connected_components(eu), key=len)
beu = eu.subgraph(bc)

In [59]:
leiden_communities(beu)

Number of communities: 8
Number of different community types: 3
['Waxman', 'Gilbert', 'Barabasi Albert']
Size of communities: [242, 147, 132, 124, 109, 97, 79, 56]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 986 nodes)
Correctly classified communities: 0
Modularity: 0.4337965152400809
Runtime:  13.342191934585571


### **Euroroad**

In [60]:
#Από ICON, ευρωπαϊκοί δρόμοι
euroroad=nx.read_edgelist("euroroad.txt",create_using=nx.Graph())
bc_e = max(nx.connected_components(euroroad), key=len)
b_e = euroroad.subgraph(bc_e)

In [61]:
leiden_communities(b_e)

Number of communities: 20
Number of different community types: 2
['Watts Strogatz', 'Erdos-Renyi']
Size of communities: [81, 75, 71, 62, 61, 59, 57, 56, 55, 54, 48, 47, 45, 44, 44, 36, 33, 31, 26, 23]
Number of discarded nodes: 31
Correctly classified nodes: 0 (out of 1008 nodes)
Correctly classified communities: 0
Modularity: 0.8718815049690991
Runtime:  1.2387499809265137


### **ChicagoRegional**

In [62]:
#Από ICON, δρόμοι στο Σικάγο
chicago=nx.read_edgelist("ChicagoRegional.txt",create_using=nx.Graph())
bc_c = max(nx.connected_components(chicago), key=len)
b_c = chicago.subgraph(bc_c)

In [63]:
leiden_communities(b_c)

Number of communities: 40
Number of different community types: 1
['Watts Strogatz']
Size of communities: [506, 481, 439, 434, 432, 411, 401, 386, 386, 376, 363, 355, 352, 347, 344, 340, 334, 323, 318, 316, 316, 315, 310, 298, 296, 293, 293, 291, 287, 284, 265, 256, 251, 241, 237, 231, 228, 226, 213, 204]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 12979 nodes)
Correctly classified communities: 0
Modularity: 0.9366705270828044
Runtime:  89.36105632781982


### **AdvogatoTrustNetwork**

In [64]:
#Από ICON, σχέσεις μεταξύ μελών του Advogato
advogato=nx.read_weighted_edgelist("advogato.txt",create_using=nx.Graph())
bc_a = max(nx.connected_components(advogato), key=len)
b_a = advogato.subgraph(bc_a)

In [65]:
leiden_communities(b_a)

Number of communities: 17
Number of different community types: 4
['Barabasi Albert', 'Waxman', 'Watts Strogatz', 'Erdos-Renyi']
Size of communities: [1135, 653, 606, 509, 445, 412, 270, 219, 136, 109, 101, 92, 82, 67, 62, 53, 23]
Number of discarded nodes: 68
Correctly classified nodes: 0 (out of 4974 nodes)
Correctly classified communities: 0
Modularity: 0.4350238670593078
Runtime:  164.15184783935547


### **Flights-FAA**

In [66]:
#Από ICON, πτήσεις του Federal Aviation Association
flights=nx.read_edgelist("flights-faa.txt",create_using=nx.Graph())
bc_f = max(nx.connected_components(flights), key=len)
b_f = flights.subgraph(bc_f)

In [67]:
leiden_communities(b_f)

Number of communities: 14
Number of different community types: 3
['Watts Strogatz', 'Barabasi Albert', 'Erdos-Renyi']
Size of communities: [147, 139, 133, 114, 114, 112, 106, 95, 95, 50, 29, 28, 26, 24]
Number of discarded nodes: 14
Correctly classified nodes: 0 (out of 1212 nodes)
Correctly classified communities: 0
Modularity: 0.7101934367521219
Runtime:  2.9370880126953125


### **Bristol**

In [68]:
#Από ICON, ΜΜΜ στο Bristol, είναι spatial
bristol = nx.read_edgelist("edges_bristol.txt", create_using=nx.Graph())
bc_b = max(nx.connected_components(bristol), key=len)
b_b = bristol.subgraph(bc_b)

In [69]:
leiden_communities(b_b)

Number of communities: 35
Number of different community types: 1
['Watts Strogatz']
Size of communities: [142, 120, 107, 106, 104, 100, 99, 95, 94, 90, 89, 86, 84, 83, 77, 75, 75, 72, 69, 68, 67, 67, 66, 65, 61, 61, 56, 46, 45, 40, 38, 38, 37, 28, 26]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 2576 nodes)
Correctly classified communities: 0
Modularity: 0.9079168092369719
Runtime:  5.295015811920166


### **Coach**

In [70]:
#Όπως το Bristol, αλλά για το Coach
coach=nx.read_edgelist("edges_coach.txt",create_using=nx.Graph())
bc_c = max(nx.connected_components(coach), key=len)
b_c = coach.subgraph(bc_c)

In [71]:
leiden_communities(b_c)

Number of communities: 36
Number of different community types: 2
['Watts Strogatz', 'Barabasi Albert']
Size of communities: [145, 141, 116, 104, 103, 90, 87, 80, 76, 75, 74, 70, 67, 66, 61, 60, 57, 56, 56, 56, 54, 54, 53, 52, 51, 51, 48, 46, 42, 41, 33, 33, 30, 28, 25, 21]
Number of discarded nodes: 100
Correctly classified nodes: 0 (out of 2302 nodes)
Correctly classified communities: 0
Modularity: 0.891170382165605
Runtime:  3.634777307510376


### **Moreno Crime**

In [72]:
#Από ICON, κοινωνικό δίκτυο εγκληματιὠν
crime = nx.read_edgelist("moreno_crime.txt", create_using=nx.Graph())
bc_c = max(nx.connected_components(crime), key=len)
b_c = crime.subgraph(bc_c)

In [73]:
leiden_communities(b_c)

Number of communities: 17
Number of different community types: 2
['Barabasi Albert', 'Watts Strogatz']
Size of communities: [75, 73, 69, 55, 49, 49, 49, 46, 44, 44, 43, 42, 40, 39, 36, 31, 25]
Number of discarded nodes: 20
Correctly classified nodes: 0 (out of 809 nodes)
Correctly classified communities: 0
Modularity: 0.6144250502729102
Runtime:  1.0087981224060059


### **Netscience**

In [74]:
#Από ICON, επιστημονικές συνεργασίες
netscience = nx.read_gml("netscience.gml")
bc_n = max(nx.connected_components(netscience), key=len)
b_n = netscience.subgraph(bc_n)

In [75]:
leiden_communities(b_n)

Number of communities: 10
Number of different community types: 2
['Barabasi Albert', 'Waxman']
Size of communities: [40, 32, 30, 30, 30, 26, 24, 24, 23, 22]
Number of discarded nodes: 98
Correctly classified nodes: 0 (out of 281 nodes)
Correctly classified communities: 0
Modularity: 0.8485029854105121
Runtime:  0.305408239364624


### **R Dependencies**

In [76]:
#Από ICON, δίκτυο εξαρτήσεων στην R
rdep=nx.Graph()
with open('dependencies.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    next(csv_reader)
    for row in csv_reader:
        rdep.add_edge(row[0],row[1])
        line_count += 1
str_to_int = {n: i for i, n in enumerate(rdep.nodes())}
rdep = nx.relabel_nodes(rdep, str_to_int)
bc_r = max(nx.connected_components(rdep), key=len)
b_r = rdep.subgraph(bc_r)

In [77]:
leiden_communities(b_r)

Number of communities: 1
Number of different community types: 1
['Gilbert']
Size of communities: [29]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 29 nodes)
Correctly classified communities: 0
Modularity: 1.1102230246251565e-16
Runtime:  0.0257875919342041


### **YuYeast**

In [78]:
yeast = nx.read_edgelist("yu_yeast.txt", create_using=nx.Graph())
bc_yy = max(nx.connected_components(yeast), key=len)
b_yy = yeast.subgraph(bc_yy)

In [79]:
leiden_communities(b_yy)

Number of communities: 24
Number of different community types: 2
['Barabasi Albert', 'Watts Strogatz']
Size of communities: [156, 134, 99, 81, 79, 79, 77, 71, 70, 69, 66, 65, 62, 61, 61, 55, 46, 45, 45, 40, 37, 36, 30, 28]
Number of discarded nodes: 55
Correctly classified nodes: 0 (out of 1592 nodes)
Correctly classified communities: 0
Modularity: 0.7535957950954273
Runtime:  2.573387622833252


### **Bacillus Megaterium**

In [80]:
bacillus_meg = pd.read_excel("BacillusMeg.xlsx", engine='openpyxl', header=None)
bacillus_meg.columns = bacillus_meg.columns.map(str)
bacillus_meg = bacillus_meg.iloc[1:]
bac_meg = nx.from_pandas_edgelist(bacillus_meg, source="1", target="2", edge_attr=True)
bc_bm = max(nx.connected_components(bac_meg), key=len)
b_bm = bac_meg.subgraph(bc_bm)
bm_map = {node: i for i, node in enumerate(b_bm.nodes())}
b_bm = nx.relabel_nodes(b_bm, bm_map)

In [81]:
leiden_communities(b_bm)

Number of communities: 1
Number of different community types: 1
['Barabasi Albert']
Size of communities: [4319]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 4319 nodes)
Correctly classified communities: 0
Modularity: 0.0
Runtime:  81.24557423591614


### **Fullerene**

In [82]:
fullerene = nx.read_gml("Fullerene C3840.gml")
bc_fu = max(nx.connected_components(fullerene), key=len)
b_fu = fullerene.subgraph(bc_fu)

In [83]:
leiden_communities(b_fu)

Number of communities: 27
Number of different community types: 1
['Watts Strogatz']
Size of communities: [204, 195, 179, 176, 170, 165, 165, 163, 158, 154, 152, 150, 145, 143, 142, 139, 134, 128, 123, 122, 122, 117, 106, 104, 102, 94, 88]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 3840 nodes)
Correctly classified communities: 0
Modularity: 0.8835814073350695
Runtime:  11.188668966293335


### **Malaria**

In [84]:
malaria = nx.read_edgelist("malaria.txt", delimiter=',')
bc_ma = max(nx.connected_components(malaria), key=len)
b_ma = malaria.subgraph(bc_ma)

In [85]:
leiden_communities(b_ma)

Number of communities: 1
Number of different community types: 1
['Random Geometric']
Size of communities: [31]
Number of discarded nodes: 81
Correctly classified nodes: 0 (out of 31 nodes)
Correctly classified communities: 0
Modularity: 0.58001282494851
Runtime:  0.06623578071594238


### **PowerNet**

In [86]:
powernet = nx.read_gml("power.gml", label='id')
bc_pn = max(nx.connected_components(powernet), key=len)
b_pn = powernet.subgraph(bc_pn)

In [87]:
leiden_communities(b_pn)

Number of communities: 40
Number of different community types: 3
['Watts Strogatz', 'Erdos-Renyi', 'Barabasi Albert']
Size of communities: [218, 210, 210, 203, 199, 198, 196, 192, 177, 177, 164, 150, 148, 147, 137, 134, 130, 129, 129, 121, 116, 116, 111, 102, 101, 95, 95, 83, 80, 75, 75, 75, 71, 67, 66, 65, 60, 51, 37, 31]
Number of discarded nodes: 0
Correctly classified nodes: 0 (out of 4941 nodes)
Correctly classified communities: 0
Modularity: 0.9387627804580388
Runtime:  13.63449740409851
